# Redis Basics Tutorial

Introduction aux bases de Redis avec notre dataset de films.

## 1. Connexion à Redis

In [1]:
import sys
sys.path.append('/home/jovyan/work')

from pprint import pprint

import redis

# Connexion à Redis via un client Redis
r = redis.Redis(host='redis', port=6379, db=0)

# Tester la connexion
print("Connection test:", r.ping())

Connection test: True


## 2. Exploration globale de la db

In [2]:
num_keys = r.dbsize()
print(f"Nombre total de clés dans Redis: {num_keys}")

Nombre total de clés dans Redis: 8354


Comme la base de donnée contient un nombre raisonnable de clés, on peut récupérer toutes les clés et les parcourir rapidement.

In [3]:
# Récupérer le type d'objet dans notre base de données

keys = r.keys('*')  # Récupérer toutes les clés
prefixes = set(key.decode('utf-8').split(':')[0] for key in keys)
print("Types d'objets dans Redis:", prefixes)

Types d'objets dans Redis: {'actor', 'theater', 'movie', 'user'}


In [4]:
# Récupérer le nombre d'objets par catégorie

category_counts = {prefix: 0 for prefix in prefixes}

for key in keys:
    category = key.decode('utf-8').split(':')[0]
    category_counts[category] += 1

print("Nombre d'objets par catégorie:", category_counts)

Nombre d'objets par catégorie: {'actor': 1319, 'theater': 117, 'movie': 922, 'user': 5996}


Vérifier le type des clés

In [6]:
# Récupérer les 10 premières clés

r.keys('*')[:10]

[b'user:4521',
 b'user:3843',
 b'user:3307',
 b'user:5795',
 b'user:697',
 b'movie:192',
 b'user:5494',
 b'user:3739',
 b'actor:463',
 b'user:499']

In [8]:
# Pour connaître le type des clés dans la base de données 

key_type = r.type('user:1421')  # Vérifie le type de la clé
print("Type de la clé:", key_type)

Type de la clé: b'hash'


## 3. Récupérer des données

### Selon le type de la clé on choisit la bonne commande 
Pour un Hash, utilisez HGET ou HGETALL.
Pour une Liste, utilisez LRANGE ou LPOP.
Pour un Set, utilisez SMEMBERS.
Pour un Sorted Set, utilisez ZRANGE. 

In [9]:
# Vérifier si la clé existe

r.exists('movie:1') == 1  # Renvoie 1 si la clé existe, 0 sinon

True

In [10]:
# Récupérer un champ spécifique d'un Hash

r.hget('movie:1', 'title')

b'Guardians of the Galaxy'

In [11]:
# Récupérer plusieurs champs d'un Hash

r.hmget('movie:1', ['title', 'genre'])

[b'Guardians of the Galaxy', b'Action']

In [12]:
# Récupérer tous les champs d'un Hash

r.hgetall('movie:1')

{b'genre': b'Action',
 b'title': b'Guardians of the Galaxy',
 b'ibmdb_id': b'tt2015381',
 b'rating': b'8.1',
 b'poster': b'https://m.media-amazon.com/images/M/MV5BMTAwMjU5OTgxNjZeQTJeQWpwZ15BbWU4MDUxNDYxODEx._V1_SX300.jpg',
 b'votes': b'704613',
 b'release_year': b'2014',
 b'plot': b'A group of intergalactic criminals must pull together to stop a fanatical warrior with plans to purge the universe.'}

## 4. Ajouter et modifier des données

HSET permet de:
- ajouter une donnée (clé - valeur) à la db si la clé spécifiée n'existe pas
- modifier une donnée si la clé spécifiée existe

HSET renvoie 1 si une donnée a été créée, 0 si une donnée a été modifié

In [ ]:
# Vérifier si la clé existe
print(f"Key exists: {r.exists('movie:1') == 1}")

# Récupérer un champ spécifique d'un Hash
print(f"Initial title: {r.hget('movie:1', 'title')}")

# Modifier un champ spécifique d'un Hash
r.hset('movie:1', 'title', 'The Shawshank Redemption')

print(f"Title after modification: {r.hget('movie:1', 'title')}")

Key exists: True
Initial title: b'Toy Story'
Title after modification: b'Toy Story'


In [ ]:
# Vérifier si la clé existe
print(f"Key exists: {r.exists('movie:6002') == 1}")

# Ajouter une donnée à la db
r.hset('movie:6002', 'title', 'Inception')

r.hgetall('movie:6002')

Key exists: False


{b'title': b'Inception'}

In [17]:
# HSET permet aussi de créer / modifier une clé avec plusieurs valeurs en une seule commande

r.hset('movie:6001', mapping={'title': 'Gladiator', 'genre': 'Action', 'year': 2000})

r.hgetall('movie:6001')

{b'title': b'Gladiator', b'genre': b'Action', b'year': b'2000'}

## 5. Suppression et expiration des données

In [ ]:
### Suppression de données 

r.hset("user:2002", mapping={"name": "Charles ", "age": "28", "city": "Marseille"})

print("Avant suppression:")
pprint(r.hgetall("user:2002"))
r.hdel('user:2002', 'age')

print('Après suppression du champ age:')
pprint(r.hgetall("user:2002"))

r.delete("user:2002")

print("Après suppression du user:")
pprint(r.hgetall("user:2002"))  

Avant suppression:
{b'age': b'28', b'city': b'Marseille', b'name': b'Charles '}
après suppression du champ age:
{b'city': b'Marseille', b'name': b'Charles '}
Après suppression du user:
{}


In [22]:
### Expiration d'une clé 

import time 

# Ajouter des données
r.hset("user:2001", mapping={"name": "Bob", "age": "25", "city": "Lyon"})

r.expire("user:2001", 5)

print("Avant expiration:")
pprint(r.hgetall("user:2001"))

time.sleep(6)

print("Après expiration:")
pprint(r.hgetall("user:2001"))

Avant expiration:
{b'age': b'25', b'city': b'Lyon', b'name': b'Bob'}
Après expiration:
{}


## 6. Pipeline

In [ ]:
# Utiliser un pipeline pour exécuter plusieurs commandes en même temps

pipe = r.pipeline()

pipe.hset('user:2572', "name", "Alice")
pipe.hset('user:2572', "age", 40)
pipe.hincrby('user:2572', "login_count", 1)
results = pipe.execute()

print(results)  
r.hmget('user:2572', ['name', 'age'])

[0, 0, 3]


[b'Alice', b'40']